# Quantization of `granite-3.3-2b-instruct` model

Recall that our overall solution uses the quantized version of the model `granite-3.3-2b-instruct`. In this lab, we will be taking in the base model `granite-3.3-2b-instruct` and quantizing it to `W4A16` - which is fixed-point integer (INT) quantization scheme for weights and floating‑point for activations - to provide both memory savings (weight - INT4) and inference acceleration (activations - BF16) with `vLLM`

**Note**: `W4A16` computation is supported on Nvidia GPUs with compute capability > 7.5 (Turing, Ampere, Ada Lovelace, Hopper).

**Note**: The steps here will take around 20-30 minutes, depending on the connectivity. The most time consuming steps are the installation of llmcompressor (up to 5 mins) and the quantization step (which can take more anywhere between 10-15 mins)

## Setting up llm-compressor first

Installing `llmcompressor` may take a minute, depending on the bandwith available. Do note the versions of `transformer` library we would be using. There is a known issue (*torch.fx.proxy.TraceError: symbolically traced variables cannot be used as inputs to control flow*) with the usage of the latest transformer library (version `4.53.2` as of July 17, 2025) in combination with the latest version of llmcompressor (version `0.6.0`).

In [9]:
!pip install -q llmcompressor==0.6.0 transformers==4.52.2


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


Let's make sure we have installed the right versions installed

In [10]:
!pip list | grep llmcompressor

llmcompressor                     0.6.0


In [11]:
!pip list | grep transformer

transformers                      4.52.2


## Let' start with the quantization of the model

There are 5 steps:
1. Loading the model
2. Preparing the calibration data
3. Applying quantization
4. Evaluation of accuracy in vLLM
5. Uploading the model to S3 (MinIO)

### Loading the model

Load the model using AutoModelForCausalLM for handling quantized saving and loading. The model can be loaded from HuggingFace directly as follows:

In [12]:
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_ID = "ibm-granite/granite-3.2-2b-instruct"
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID, device_map="auto", torch_dtype="auto",
)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Prepare calibration data

Prepare the calibration data. When quantizing weigths of a model to int4 using GPTQ, we need some sample data to run the GPTQ algorithms. As a result, it is very important to use calibration data that closely matches the type of data used in our deployment. If you have fine-tuned a model, using a sample of your training data is a good idea.

In our case, we are quantizing an Instruction tuned generic model, so we will use the ultrachat dataset. Some best practices include:
- 512 samples is a good place to start (increase if accuracy drops). We are going to use 256 to speed up the process.
- 2048 sequence length is a good place to start
- Use the chat template or instrucion template that the model is trained with


In [23]:
from datasets import load_dataset

NUM_CALIBRATION_SAMPLES = 512  # 1024
DATASET_ID = "neuralmagic/LLM_compression_calibration"
DATASET_SPLIT = "train"

# Load dataset.
ds = load_dataset(DATASET_ID, split=DATASET_SPLIT)
ds = ds.shuffle(seed=42).select(range(NUM_CALIBRATION_SAMPLES))

# Preprocess the data into the format the model is trained with.
def preprocess(example):
    return {"text": example["text"]}
ds = ds.map(preprocess)

# Tokenize the data
def tokenize(sample):
    return tokenizer(
        sample["text"],
        padding=False,
        truncation=False,
        add_special_tokens=True,
    )
ds = ds.map(tokenize, remove_columns=ds.column_names)

With the dataset ready, we will now apply quantization.

We first select the quantization algorithm. For W4A16, we want to:
- Run SmoothQuant to make the activations easier to quantize
- Quantize the weights to 4 bits with channelwise scales using GPTQ
- Quantize the activations with dynamic per token strategy

**Note**: The quantization step takes a long time to complete due to the callibration requirements -- around 10 - 15 mins, depending on the GPU.

### Imports and definitions

**GPTQModifier**: Applies Gentle Quantization (GPTQ) for weight-only quantization.

**SmoothQuantModifier**: Prepares model activations for smoother quantization by scaling internal activations and weights.

**oneshot**: High-level API that applies your quantization recipe in one go.

In [24]:
from llmcompressor.modifiers.quantization import GPTQModifier
from llmcompressor.transformers import oneshot
from llmcompressor.modifiers.smoothquant import SmoothQuantModifier

### Hyperparameters

Rationale
- **DAMPENING_FRAC=0.1** gently prevents large Hessian-derived updates during quantization.
- **OBSERVER="mse"** measures quantization error by squared deviations, yielding well-rounded scales.
- **GROUP_SIZE=128** determines group size for per-channel quantization; typical default usage.

In [25]:
DAMPENING_FRAC = 0.1  # tapering adjustment to prevent extreme weight updates
OBSERVER = "mse"  # denotes minmax - quantization layout based on mean‐squared‐error
GROUP_SIZE = 128  # # per-channel grouping width for quantization

### Layer Mappings & Ignoring Heads

Logic

- **ignore=["lm_head"]** skips quantization on the output layer to preserve final logits and maintain accuracy.
- mappings link groups of linear projections (q, k, v, gating, up/down projections) with layernorm blocks—SmoothQuant uses these to shift and normalize activations across paired layers for better quant distribution.

In [26]:
ignore=["lm_head"]
mappings=[
    [["re:.*q_proj", "re:.*k_proj", "re:.*v_proj"], "re:.*input_layernorm"],
    [["re:.*gate_proj", "re:.*up_proj"], "re:.*post_attention_layernorm"],
    [["re:.*down_proj"], "re:.*up_proj"]
]

### Recipe Definition

**Workflow**

- **SmoothQuantModifier**: Re-scales activations across paired layers before quantization to reduce outliers (smoothing_strength=0.7, high smoothing but not extreme).
- **GPTQModifier**: Performs Weight-Only quantization (4-bit weights, 16-bit activations) on all Linear layers except those ignored, applying your dampening and observer settings. Scheme "W4A16" reduces model size while maintaining decent accuracy. 

In [27]:
recipe = [
    SmoothQuantModifier(smoothing_strength=0.7, ignore=ignore, mappings=mappings),
    GPTQModifier(
        targets=["Linear"],
        ignore=ignore,
        scheme="W4A16",
        dampening_frac=DAMPENING_FRAC,
        observer=OBSERVER,
    )
]

### Quantize in One Shot

**How It Works**

- Feeds dataset (calibration set) into your model to gather activation statistics.
- Applies SmoothQuant rescaling followed by GPTQ quantization in a sequential per-layer manner.
- **max_seq_length=8196** ensures large context coverage for calibration.

In [28]:
oneshot(
    model=model,
    dataset=ds,
    recipe=recipe,
    num_calibration_samples=NUM_CALIBRATION_SAMPLES,
    max_seq_length=8196,
)

/tmp/ipykernel_2728/180673739.py:1: DeprecationWarning: `from llmcompressor.transformers import oneshot` is deprecated, please use `from llmcompressor import oneshot`.
  oneshot(


2025-07-17T04:25:03.079329+0000 | reset | INFO - Compression lifecycle reset
2025-07-17T04:25:03.082644+0000 | from_modifiers | INFO - Creating recipe from modifiers
2025-07-17T04:25:04.425764+0000 | initialize | INFO - Compression lifecycle initialized for 1 modifiers
2025-07-17T04:25:04.426533+0000 | IndependentPipeline | INFO - Inferred `SequentialPipeline` for `SmoothQuantModifier`


(1/41): Calibrating: 100%|██████████| 512/512 [00:01<00:00, 303.10it/s]

2025-07-17T04:25:07.961550+0000 | _apply_smoothing | INFO - Smoothing with model.layers.0.input_layernorm
2025-07-17T04:25:08.002475+0000 | _apply_smoothing | INFO - Smoothing with model.layers.0.post_attention_layernorm
2025-07-17T04:25:08.004240+0000 | _apply_smoothing | INFO - Smoothing with model.layers.0.mlp.up_proj



(2/41): Calibrating: 100%|██████████| 512/512 [00:01<00:00, 463.90it/s]

...

(40/41): Calibrating: 100%|██████████| 512/512 [00:07<00:00, 66.77it/s]

2025-07-17T04:38:16.092048+0000 | compress_modules | INFO - Quantizing model.layers.39.self_attn.q_proj using 512 samples


2025-07-17T04:38:16.948643+0000 | compress | METRIC - time 0.85s
2025-07-17T04:38:16.949703+0000 | compress | METRIC - error 68646.34
2025-07-17T04:38:16.950535+0000 | compress | METRIC - GPU 0 | usage: 25.08% | total memory: 24 GB
2025-07-17T04:38:16.951046+0000 | compress | METRIC - Compressed module size: 8.486912 MB
2025-07-17T04:38:16.951953+0000 | compress_modules | INFO - Quantizing model.layers.39.self_attn.k_proj using 512 samples
2025-07-17T04:38:17.808140+0000 | compress | METRIC - time 0.86s
2025-07-17T04:38:17.809307+0000 | compress | METRIC - error 18900.18
2025-07-17T04:38:17.810247+0000 | compress | METRIC - GPU 0 | usage: 25.08% | total memory: 24 GB
2025-07-17T04:38:17.810687+0000 | compress | METRIC - Compressed module size: 2.121728 MB
2025-07-17T04:38:17.811460+0000 | compress_modules | INFO - Quantizing model.layers.39.self_attn.v_proj using 512 samples
2025-07-17T04:38:18.653772+0000 | compress | METRIC - time 0.84s
2025-07-17T04:38:18.654980+0000 | compress | ME

(41/41): Propagating: 100%|██████████| 512/512 [00:01<00:00, 297.61it/s]


2025-07-17T04:38:29.664843+0000 | finalize | INFO - Compression lifecycle finalized for 1 modifiers
2025-07-17T04:38:30.240023+0000 | post_process | WARNING - Optimized model is not saved. To save, please provide`output_dir` as input arg.Ex. `oneshot(..., output_dir=...)`


### Save the Compressed Model

**Explanation**

- Naming: appends -W4A16 to distinguish the quantized checkpoint.
- **save_compressed=True** stores weights in compact safetensors format for deployment via vLLM.

In [29]:
# Save to disk compressed.
SAVE_DIR = MODEL_ID.split("/")[-1] + "-W4A16"
model.save_pretrained(SAVE_DIR, save_compressed=True)
tokenizer.save_pretrained(SAVE_DIR)

2025-07-17T04:41:16.812481+0000 | get_model_compressor | INFO - skip_sparsity_compression_stats set to True. Skipping sparsity compression statistic calculations. No sparsity compressor will be applied.


Compressing model: 527it [00:12, 43.02it/s]


('granite-3.2-2b-instruct-W4A16/tokenizer_config.json',
 'granite-3.2-2b-instruct-W4A16/special_tokens_map.json',
 'granite-3.2-2b-instruct-W4A16/chat_template.jinja',
 'granite-3.2-2b-instruct-W4A16/vocab.json',
 'granite-3.2-2b-instruct-W4A16/merges.txt',
 'granite-3.2-2b-instruct-W4A16/added_tokens.json',
 'granite-3.2-2b-instruct-W4A16/tokenizer.json')

### Evaluate accuracy in vLLM

We can evaluate accuracy with lm_eval

##### Check GPU memory leftovers:

In [1]:
!nvidia-smi

Thu Jul 17 04:42:13 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.148.08             Driver Version: 570.148.08     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      On  |   00000000:31:00.0 Off |                    0 |
| N/A   50C    P8             17W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

**IMPORTANT**: After quantizing the model the GPU memory may not be freed (see the above output). You need to **restart the kernel** before evaluating the model to ensure you have enough GPU RAM available.

#### Install lm_eval

In [2]:
!pip install -q lm_eval==v0.4.3


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


#### Install vLLM for evaluation

Run the following to test accuracy on GSM-8K:

In [3]:
pip install -q vllm

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmcompressor 0.6.0 requires numpy<2.0,>=1.17.0, but you have numpy 2.2.6 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Evaluation Command

- `--model vllm` - Uses vLLM backend for fast, memory-efficient inference on large models 
- `--model_args` - pretrained=$MODEL_ID: specifies which model to load.
- `add_bos_token=true`: ensures a beginning-of-sequence token is added; required for consistent results on math and reasoning tasks 
- `max_model_len=4096`: sets the context window the model uses for evaluation.
- `gpu_memory_utilization=0.5`: limits vLLM to use 50% of GPU memory, allowing to avoid OOM.

In [9]:
import os

current_dir = os.getcwd()

MODEL_ID = current_dir + "/granite-3.2-2b-instruct-W4A16"

!lm_eval --model vllm \
  --model_args "pretrained=$MODEL_ID,add_bos_token=true,max_model_len=4096,gpu_memory_utilization=0.5" \
  --trust_remote_code \
  --tasks gsm8k \
  --num_fewshot 5 \
  --limit 250 \
  --batch_size 'auto'

INFO 07-17 04:50:30 [__init__.py:244] Automatically detected platform cuda.
2025-07-17:04:50:32,416 INFO     [__main__.py:272] Verbosity set to INFO
2025-07-17:04:50:36,510 WARNING  [__main__.py:312]  --limit SHOULD ONLY BE USED FOR TESTING.REAL METRICS SHOULD NOT BE COMPUTED USING LIMIT.
2025-07-17:04:50:36,511 INFO     [__main__.py:357] Passed `--trust_remote_code`, setting environment variable `HF_DATASETS_TRUST_REMOTE_CODE=true`
2025-07-17:04:50:36,511 INFO     [__main__.py:369] Selected Tasks: ['gsm8k']
2025-07-17:04:50:36,513 INFO     [evaluator.py:152] Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234
2025-07-17:04:50:36,513 INFO     [evaluator.py:189] Initializing vllm model, with arguments: {'pretrained': '/opt/app-root/src/showroom-summit2025-lb2959-neural-magic/lab-materials/03/granite-3.2-2b-instruct-W4A16', 'add_bos_token': True, 'max_model_len': 4096, 'gpu_memory_utilization': 0.5, 'trust_remote_code': True}
INFO 07-17 04:50:43 [con

With powerful GPU(s), you could also run the vLLM based evals with the following - using higher GPU memory utilization and chunked prefill. 
```bash
!lm_eval \
  --model vllm \
  --model_args pretrained=$SAVE_DIR,dtype=auto,add_bos_token=True,max_model_len=4096,tensor_parallel_size=1,gpu_memory_utilization=0.8,enable_chunked_prefill=True \
  --trust_remote_code \
  --tasks openllm \
  --write_out \
  --batch_size auto \
  --output_path output_dir \
  --show_config
```

**Next Steps**: 
- How would you futher improve the accuacy of the model?
- How would you go about preparing the right data set for a different use case?

### Upload Optimized Model to MinIO

In [2]:
import os
from boto3 import client

current_dir = os.getcwd()
OPTIMIZED_MODEL_DIR = current_dir + "/granite-3.2-2b-instruct-W4A16"
S3_PATH = "granite-int4-notebook"

print('Starting upload of quantizied model')
s3_endpoint_url = os.environ["AWS_S3_ENDPOINT"]
s3_access_key = os.environ["AWS_ACCESS_KEY_ID"]
s3_secret_key = os.environ["AWS_SECRET_ACCESS_KEY"]
s3_bucket_name = os.environ["AWS_S3_BUCKET"]

print(f'Uploading predictions to bucket {s3_bucket_name} '
        f'to S3 storage at {s3_endpoint_url}')

s3_client = client(
    's3', endpoint_url=s3_endpoint_url, aws_access_key_id=s3_access_key,
    aws_secret_access_key=s3_secret_key, verify=False
)

# Walk through the local folder and upload files
for root, dirs, files in os.walk(OPTIMIZED_MODEL_DIR):
    for file in files:
        local_file_path = os.path.join(root, file)
        s3_file_path = os.path.join(S3_PATH, local_file_path[len(OPTIMIZED_MODEL_DIR)+1:])
        s3_client.upload_file(local_file_path, s3_bucket_name, s3_file_path)
        print(f'Uploaded {local_file_path}')

print('Finished uploading of quantizied model')

Starting results upload.
Uploading predictions to bucket models to S3 storage at http://minio-service.minio.svc.cluster.local:9000
Uploaded /opt/app-root/src/showroom-summit2025-lb2959-neural-magic/lab-materials/03/granite-3.2-2b-instruct-W4A16/merges.txt
Uploaded /opt/app-root/src/showroom-summit2025-lb2959-neural-magic/lab-materials/03/granite-3.2-2b-instruct-W4A16/model.safetensors
Uploaded /opt/app-root/src/showroom-summit2025-lb2959-neural-magic/lab-materials/03/granite-3.2-2b-instruct-W4A16/added_tokens.json
Uploaded /opt/app-root/src/showroom-summit2025-lb2959-neural-magic/lab-materials/03/granite-3.2-2b-instruct-W4A16/tokenizer_config.json
Uploaded /opt/app-root/src/showroom-summit2025-lb2959-neural-magic/lab-materials/03/granite-3.2-2b-instruct-W4A16/chat_template.jinja
Uploaded /opt/app-root/src/showroom-summit2025-lb2959-neural-magic/lab-materials/03/granite-3.2-2b-instruct-W4A16/config.json
Uploaded /opt/app-root/src/showroom-summit2025-lb2959-neural-magic/lab-materials/03/